In [55]:
import tensorflow as tf

import numpy as np
import os
import time

from nltk.corpus import gutenberg
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# shakespeare corpus
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
# gutenberg corpus
# fileids_ = gutenberg.fileids()

In [56]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [57]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters', vocab)

65 unique characters ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [58]:
# 텍스트 처리 - 벡터화 및 문자열 복구
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [59]:
# get train sample
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [60]:
seq_length = 110
slide_window = 7
examples_per_epoch = len(text)//(seq_length+slide_window)

In [61]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

def split_input_target(sequence):
    input_text = sequence[:-1*slide_window]
    target_text = sequence[slide_window:]
    return input_text, target_text

In [83]:
sequences = ids_dataset.batch(seq_length+slide_window, drop_remainder=True)
dataset = sequences.map(split_input_target)
# dataset = dataset[:6000]
# validate = dataset[6001:]


TypeError: 'MapDataset' object is not subscriptable

(7149,
                                                       0  \
 7211  (tf.Tensor(7, shape=(), dtype=int64), tf.Tenso...   
 5340  (tf.Tensor(48, shape=(), dtype=int64), tf.Tens...   
 4539  (tf.Tensor(57, shape=(), dtype=int64), tf.Tens...   
 7458  (tf.Tensor(28, shape=(), dtype=int64), tf.Tens...   
 7044  (tf.Tensor(45, shape=(), dtype=int64), tf.Tens...   
 ...                                                 ...   
 8742  (tf.Tensor(1, shape=(), dtype=int64), tf.Tenso...   
 6505  (tf.Tensor(59, shape=(), dtype=int64), tf.Tens...   
 4707  (tf.Tensor(2, shape=(), dtype=int64), tf.Tenso...   
 2701  (tf.Tensor(7, shape=(), dtype=int64), tf.Tenso...   
 5255  (tf.Tensor(44, shape=(), dtype=int64), tf.Tens...   
 
                                                       1  
 7211  (tf.Tensor(45, shape=(), dtype=int64), tf.Tens...  
 5340  (tf.Tensor(2, shape=(), dtype=int64), tf.Tenso...  
 4539  (tf.Tensor(40, shape=(), dtype=int64), tf.Tens...  
 7458  (tf.Tensor(57, shape=(), dty

In [ ]:
# Make train Batch
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
# set model
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, mode='gru'):
        super().__init__(self)
        self.mode = mode
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.rnn = tf.keras.layers.SimpleRNN(rnn_units, return_sequences=True, return_state=True) if mode == 'simple_rnn' \
            else tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True) if mode == 'lstm' \
            else tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.rnn.get_initial_state(x)

        if self.mode == 'lstm':
            x, final_memory_state, final_carry_state = self.rnn(x, initial_state=states, training=training)
        else:
            x, state = self.rnn(x, initial_state=states, training=training)

        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
# mode의 default 값은 'gru'
model = MyModel(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    # mode = 'simple_rnn'
    mode = 'lstm')

In [ ]:
# embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(dataset,training=False)
#
#
# rnn, state = tf.keras.layers.SimpleRNN(rnn_units, return_sequences=True, return_state=True)(embedding)
# rnn = tf.keras.layers.Dense(vocab_size)(rnn)
#
# lstm, final_memory_state, final_carry_state = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)(embedding)
# lstm = tf.keras.layers.Dense(vocab_size)(lstm)
# gru, state = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)(embedding)
# gru = tf.keras.layers.Dense(vocab_size)(gru)
# # concate_ = tf.keras.layers.Concatenate()([rnn, lstm, gru])
# # dense = tf.keras.layers.Dense(vocab_size)(concate_)
#
# model_rnn = tf.keras.Model(dataset, rnn)
# model_lstm = tf.keras.Model(dataset, lstm)
# model_gru = tf.keras.Model(dataset, gru)

In [63]:
# train model
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
# voting.compile(optimizer='adam', loss=loss)

In [64]:
# check point
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3, restore_best_weights=True)

In [65]:
# 저장된 ckpt 읽어서 이어서 학습하기
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)
# voting.load_weights(latest)

AttributeError: 'NoneType' object has no attribute 'endswith'

In [ ]:
EPOCHS = 1
history = model.fit(dataset, epochs=EPOCHS, callbacks=[earlystopping_callback, checkpoint_callback], validation_data=validate)

In [ ]:
# plt.plot(loss)
# plt.show()

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

In [ ]:
# model save
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')